# Low-Pass Filtering IMU Sample Data
This notebook loads the synthetic Sensor data from the Seeed Studio XIAO nRF52840 Sense Plus, applies a simple first-order low-pass filter to the motion signals, and visualizes the noise reduction.

## Imports and Configuration
We'll use pandas/numpy for data wrangling and Matplotlib for visualization.

In [ ]:
from fitness_ai import processed_data_path, raw_data_path

RAW_DATA_PATH = raw_data_path('deep-squat/athlete_01_deep-squat_round1.csv')
OUTPUT_PATH = processed_data_path('deep-squat/athlete_01_deep-squat_round1.csv')

OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f'Raw data: {RAW_DATA_PATH}')
print(f'Filtered output will be written to: {OUTPUT_PATH}')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8')


## Load the IMU Sample
The CSV stores 10 seconds of data sampled at 50 Hz.

In [ ]:
FS = 100.0  # sampling frequency in Hz for the wrist IMU

df = pd.read_csv(RAW_DATA_PATH)
df.head()


## Define a Low-Pass Filter
We implement a first-order infinite impulse response (IIR) low-pass filter based on an RC network. It attenuates frequencies above the chosen cutoff while preserving lower-frequency motion.

In [ ]:
def low_pass_filter(signal: np.ndarray, cutoff_hz: float, fs: float) -> np.ndarray:
    """Apply a simple first-order low-pass filter."""
    dt = 1.0 / fs
    rc = 1.0 / (2 * np.pi * cutoff_hz)
    alpha = dt / (rc + dt)
    filtered = np.zeros_like(signal)
    filtered[0] = signal[0]
    for i in range(1, len(signal)):
        filtered[i] = filtered[i - 1] + alpha * (signal[i] - filtered[i - 1])
    return filtered

cutoff_hz = 5.0  # adjust based on how aggressive you want the smoothing
columns_to_filter = [
    'accel_x_ms2', 'accel_y_ms2', 'accel_z_ms2',
    'gyro_x_dps', 'gyro_y_dps', 'gyro_z_dps'
]


filtered_df = df.copy()
for column in columns_to_filter:
    filtered_df[column] = low_pass_filter(df[column].to_numpy(), cutoff_hz, FS)


## Compare Raw vs Filtered Signals
We'll plot accelerometer and gyroscope data to see the smoothing effect of the filter.

In [ ]:
time_s = df['loop'] / FS

axis_configs = [
    ('accel_x_ms2', 'Acceleration X (m/s^2)', 'Accelerometer X'),
    ('accel_y_ms2', 'Acceleration Y (m/s^2)', 'Accelerometer Y'),
    ('accel_z_ms2', 'Acceleration Z (m/s^2)', 'Accelerometer Z'),
    ('gyro_x_dps', 'Angular velocity X (deg/s)', 'Gyroscope X'),
    ('gyro_y_dps', 'Angular velocity Y (deg/s)', 'Gyroscope Y'),
    ('gyro_z_dps', 'Angular velocity Z (deg/s)', 'Gyroscope Z'),
]

fig, axes = plt.subplots(3, 2, figsize=(16, 10), sharex=True)
axes = axes.flatten()

for ax, (column, ylabel, title) in zip(axes, axis_configs):
    ax.plot(time_s, df[column], label='Raw', alpha=0.4)
    ax.plot(time_s, filtered_df[column], label='Filtered', linewidth=1.8)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend(loc='upper right')

for ax in axes[-2:]:
    ax.set_xlabel('Time (s)')

fig.tight_layout()
plt.show()


## Save Filtered Output (Optional)
You can persist the filtered signals for downstream processing.

In [ ]:
filtered_df.to_csv(OUTPUT_PATH, index=False)
OUTPUT_PATH
